### Tutorial de transformers de Hugging Face 

Este cuaderno proporcionará una introducción a la librería Python Hugging Face Transformers y algunos patrones comunes que puedes usar para aprovecharla. Es más útil para usar o ajustar modelos transformadores preentrenados para tus proyectos.

Hugging Face proporciona acceso a modelos (tanto el código que los implementa como sus pesos preentrenados, incluyendo los últimos LLMs como Llama3, DBRX, etc.), tokenizadores específicos de los modelos, así como pipelines para tareas comunes de NLP, y datasets y métricas en un paquete separado llamado `datasets`. Tiene implementaciones en PyTorch, Tensorflow y Flax (¡aunque usaremos las versiones de PyTorch aquí!)

Vamos a repasar algunos casos de uso:
* Descripción general de Tokenizers y Modelos
* Ajuste fino - para tu propia tarea. Usaremos un ejemplo de clasificación de sentimientos.

Se pueden aplicar a otros proyectos interesantes tambien:

1. Aplicar un modelo preentrenado existente a una nueva aplicación o tarea y explorar cómo abordarlo/solucionarlo.
2. Implementar una nueva o compleja arquitectura neural y demostrar su rendimiento en algunos datos.
3. Analizar el comportamiento de un modelo: cómo representa el conocimiento lingüístico o qué tipo de fenómenos puede manejar o errores que comete.

De estos, `transformers` será de mayor ayuda para (1) y para (3). (2) implica una curva de aprendizaje, pero si la dominas, encontrarás muy conveniente diseñar un modelo basado en los existentes proporcionados por Hugging Face. No lo cubriremos aquí y por favor refiérete a [este ejemplo](https://huggingface.co/docs/transformers/en/custom_models).

Aquí hay recursos adicionales que introducen la librería que se utilizaron para hacer este cuaderno:

* [Docs de Hugging Face](https://huggingface.co/docs/transformers/index)
  * Documentación clara
  * Tutoriales, recorridos y cuadernos de ejemplo
  * Lista de modelos disponibles
* [Curso de Hugging Face](https://huggingface.co/course/)
* [Ejemplos de Hugging Face](https://github.com/huggingface/transformers/tree/main/examples/pytorch) Puedes encontrar estructuras de código muy similares en tareas/modelos descendentes muy diferentes usando Hugging Face.


In [ ]:
# Instalación de las bibliotecas necesarias
!pip install transformers
!pip install datasets
!pip install accelerate

Se escribe una función print_encoding diseñada para imprimir de manera legible el contenido de un diccionario,para mostrar las entradas del modelo después de la tokenización. 

In [ ]:
from collections import defaultdict, Counter
import json

from matplotlib import pyplot as plt
import numpy as np
import torch

def print_encoding(model_inputs, indent=4):
    indent_str = " " * indent
    print("{")
    for k, v in model_inputs.items():
        print(indent_str + k + ":")
        print(indent_str + indent_str + str(v))
    print("}")

### 1. Patrón común para usar Transformers de Hugging Face

Vamos a empezar con un patrón de uso común para Transformadores de Hugging Face, usando el ejemplo de análisis de sentimientos.

Primero, encuentra un modelo en el [hub](https://huggingface.co/models) de Hugging Face. Cualquiera puede subir su modelo para que otras personas lo usen. (Estoy usando un modelo de análisis de sentimientos de [este artículo](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3489963)).

Luego, hay dos objetos que necesitan ser inicializados: un **tokenizador** y un **modelo**

* El tokenizador convierte cadenas en listas de IDs de vocabulario que el modelo requiere.
* El modelo toma los IDs de vocabulario y produce una predicción.

![full_nlp_pipeline.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)
De [https://huggingface.co/course/chapter2/2?fw=pt](https://huggingface.co/course/chapter2/2?fw=pt)



#### RoBERTa

RoBERTa (Robustly optimized BERT approach) es un modelo de lenguaje preentrenado desarrollado por Facebook AI. Es una variante del modelo BERT (Bidirectional Encoder Representations from Transformers) con algunas mejoras en el entrenamiento que lo hacen más robusto y eficaz en diversas tareas de procesamiento de lenguaje natural (NLP).


- Preentrenamiento con más datos: RoBERTa se entrena con más datos que BERT, lo que mejora su capacidad para capturar patrones y relaciones en el lenguaje.
- Más pasos de entrenamiento: Realiza más pasos de entrenamiento para mejorar el aprendizaje del modelo.
- Batch sizes más grandes: Utiliza lotes de datos más grandes durante el entrenamiento, lo que ayuda a estabilizar y mejorar el aprendizaje.
- Sin enmascaramiento de próxima oración: A diferencia de BERT, RoBERTa elimina la tarea de predicción de la próxima oración, lo que simplifica el entrenamiento y se enfoca más en la predicción de palabras enmascaradas.

El código siguiente utiliza el modelo RoBERTa preentrenado para la clasificación de secuencias, específicamente para la clasificación de sentimientos en inglés.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Inicializar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
# Inicializar el modelo
modelo = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")


In [ ]:
inputs = "I'm  happy to learn about Hugging Face Transformers!"
tokenized_inputs = tokenizer(inputs, return_tensors="pt")
outputs = modelo(**tokenized_inputs)

labels = ['NEGATIVE', 'POSITIVE']
prediction = torch.argmax(outputs.logits)


print("Entradas:")
print(inputs)
print()
print("Tokenized Inputs:")
print_encoding(tokenized_inputs)
print()
print("Salida del modelo:")
print(outputs)
print()
print(f"La prediccion es {labels[prediction]}")

### 1.2 Tokenizers (Tokenizadores)

Los modelos preentrenados se implementan junto con **tokenizadores** que se usan para preprocesar sus entradas. Los tokenizadores toman cadenas de texto o listas de cadenas y producen lo que son efectivamente diccionarios que contienen las entradas del modelo.

Puedes acceder a los tokenizadores ya sea con la clase Tokenizer específica del modelo que deseas usar (aquí DistilBERT), o con la clase AutoTokenizer.
Los Fast Tokenizers están escritos en Rust, mientras que sus versiones lentas están escritas en Python.


In [ ]:
from transformers import DistilBertTokenizer, DistilBertTokenizerFast, AutoTokenizer
name = "distilbert/distilbert-base-cased"

#### DistilBERT

DistilBERT es una versión comprimida y optimizada del modelo BERT (Bidirectional Encoder Representations from Transformers). Fue desarrollado por Hugging Face con el objetivo de hacer que los modelos de lenguaje grandes sean más ligeros, rápidos y eficientes sin una pérdida significativa de rendimiento. DistilBERT se entrena utilizando un proceso llamado distillation (destilación), en el que un modelo más pequeño (el estudiante) aprende a reproducir el comportamiento de un modelo más grande (el maestro).

**Características principales de DistilBERT**

- Tamaño reducido: DistilBERT tiene aproximadamente la mitad de los parámetros de BERT base, lo que lo hace más ligero y fácil de desplegar en producción.
- Velocidad: Debido a su menor tamaño, DistilBERT es más rápido tanto en entrenamiento como en inferencia.
- Rendimiento: A pesar de ser más pequeño y rápido, DistilBERT mantiene alrededor del 97% de la precisión de BERT en una variedad de tareas de procesamiento de lenguaje natural.

DistilBERT se entrena utilizando un método llamado destilación de conocimiento, que implica entrenar un modelo más pequeño para imitar el comportamiento de un modelo más grande. El proceso incluye:

- Entrenamiento con pérdidas combinadas: El modelo estudiante se entrena con una combinación de la pérdida estándar (por ejemplo, pérdida de entropía cruzada) y la pérdida de distilación, que mide qué tan bien las predicciones del modelo estudiante coinciden con las del modelo maestro.
- Conservación del conocimiento: El modelo estudiante aprende a conservar y replicar el conocimiento adquirido por el modelo maestro, pero de una manera más compacta y eficiente.

El siguiente código muestra cómo inicializar y utilizar DistilBERT para tareas de tokenización:

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(name)      # escrito en Python
print(tokenizer)
tokenizer = DistilBertTokenizerFast.from_pretrained(name)  # escrito en Rust
print(tokenizer)
tokenizer = AutoTokenizer.from_pretrained(name) # por defecto es Fast
print(tokenizer)

Este resultado muestra la configuración y las propiedades de diferentes tokenizadores de DistilBERT que se han inicializado con el nombre de modelo distilbert/distilbert-base-cased.

```
DistilBertTokenizer(
    name_or_path='distilbert/distilbert-base-cased',
    vocab_size=28996,
    model_max_length=512,
    is_fast=False,
    padding_side='right',
    truncation_side='right',
    special_tokens={
        'unk_token': '[UNK]',
        'sep_token': '[SEP]',
        'pad_token': '[PAD]',
        'cls_token': '[CLS]',
        'mask_token': '[MASK]'
    },
    clean_up_tokenization_spaces=True
)
```

DistilBertTokenizerFast (dos veces con la misma configuración).

```
DistilBertTokenizerFast(
    name_or_path='distilbert/distilbert-base-cased',
    vocab_size=28996,
    model_max_length=512,
    is_fast=True,
    padding_side='right',
    truncation_side='right',
    special_tokens={
        'unk_token': '[UNK]',
        'sep_token': '[SEP]',
        'pad_token': '[PAD]',
        'cls_token': '[CLS]',
        'mask_token': '[MASK]'
    },
    clean_up_tokenization_spaces=True
)
```

El resultado muestra que has inicializado tres tokenizadores para el modelo distilbert-base-cased:

- DistilBertTokenizer: Este es el tokenizador estándar, escrito en Python, que no es tan rápido como su contraparte rápida, pero aún es ampliamente utilizado para tareas de NLP.
- DistilBertTokenizerFast (dos veces con la misma configuración): Estos son los tokenizadores rápidos, escritos en Rust, que son más eficientes en términos de tiempo de ejecución. Aunque se muestran dos veces, ambos representan la misma configuración y funcionalidad, indicando que has inicializado el tokenizador rápido más de una vez.

El código siguiente proporciona una demostración de cómo tokenizar una cadena de texto utilizando un tokenizador preentrenado de Hugging Face.

La salida incluye los identificadores de los tokens y la máscara de atención, que son esenciales para el procesamiento por parte del modelo. Además, se demuestra cómo acceder a los tokens de dos maneras diferentes, resaltando la flexibilidad de los objetos devueltos por la biblioteca transformers.

**Máscara de atención**

Una máscara de atención (attention mask) es una herramienta utilizada en modelos de procesamiento de lenguaje natural, especialmente en arquitecturas de transformers, para indicar qué tokens (palabras o subpalabras) deben ser atendidos por el modelo y cuáles deben ser ignorados durante el proceso de atención.

En el contexto de los transformers y modelos como BERT o DistilBERT, las secuencias de entrada suelen tener diferentes longitudes. Sin embargo, para procesarlas de manera eficiente en lotes (batches), las secuencias deben ser de la misma longitud. Esto se logra mediante el relleno (padding), que añade tokens especiales ([PAD]) al final de las secuencias más cortas para que todas alcancen la misma longitud. La máscara de atención se utiliza para asegurarse de que estos tokens de relleno no influyan en las predicciones del modelo.

Algunas función de la máscara de atención son:

- Indicar tokens relevantes: La máscara de atención señala qué tokens en la secuencia son relevantes y deben ser considerados por el modelo.
- Ignorar tokens de relleno: La máscara de atención asegura que los tokens de relleno ([PAD]) añadidos a las secuencias más cortas sean ignorados durante el cálculo de la atención.

La máscara de atención es una lista o tensor de la misma longitud que la secuencia de entrada tokenizada. Contiene valores binarios:

- 1: Indica que el token correspondiente es relevante y debe ser atendido.
- 0: Indica que el token correspondiente es un token de relleno y debe ser ignorado.



In [ ]:
# Así es como llamas al tokenizador
input_str = "Hugging Face Transformers is great!"
tokenized_inputs = tokenizer(input_str) # https://huggingface.co/learn/nlp-course/en/chapter6/6

print("Tokenización Básica")
print_encoding(tokenized_inputs)
print()

# Dos formas de acceder:
print(tokenized_inputs.input_ids)
print(tokenized_inputs["input_ids"])

El código siguiente realiza una serie de pasos para tokenizar una cadena de texto, agregar tokens especiales (como los tokens de clasificación `[CLS]` y separación `[SEP])`, y luego decodificar los tokens de vuelta a texto.

Estos pasos no crean la máscara de atención ni añaden los caracteres especiales.

In [ ]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

# La tokenización ocurre en unos pocos pasos:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("inicio:                ", input_str)
print("tokenizar:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("agregar tokens especiales:   ", input_ids_special_tokens)
print("--------")
print("decodificar:               ", decoded_str)


El siguiente fragmento de código utiliza el FastTokenizer de la biblioteca transformers para tokenizar una cadena de texto y luego analiza los tokens resultantes en detalle. 

In [ ]:
# Para Fast Tokenizer, hay otra opción también:
inputs = tokenizer._tokenizer.encode(input_str)

print(input_str)
print("-"*5)
print(f"Número de tokens: {len(inputs)}")
print(f"Ids: {inputs.ids}")
print(f"Tokens: {inputs.tokens}")
print(f"Máscara de tokens especiales: {inputs.special_tokens_mask}")
print()
print("char_to_word da el wordpiece de un carácter en la entrada")
char_idx = 8
print(f"Por ejemplo, el {char_idx + 1}º carácter de la cadena es '{input_str[char_idx]}',"+\
      f" y es parte del wordpiece {inputs.char_to_token(char_idx)}, '{inputs.tokens[inputs.char_to_token(char_idx)]}'")


In [ ]:
# Otros trucos interesantes:
# El tokenizador puede devolver tensores de pytorch
model_inputs = tokenizer("¡Los Transformadores de Hugging Face son geniales!", return_tensors="pt")
print("Tensores PyTorch:")
print_encoding(model_inputs)

El código siguiente demuestra cómo tokenizar y rellenar múltiples secuencias de texto para que tengan la misma longitud, lo cual es necesario para el procesamiento por lotes en modelos de transformers. También se muestra cómo se utilizan los tokens de relleno (padding) y las máscaras de atención para indicar qué partes de las secuencias son relevantes para el modelo. Esta técnica asegura que los modelos de lenguaje puedan procesar secuencias de longitud variable de manera eficiente y precisa.

In [ ]:
model_inputs = tokenizer(["Hugging Face Transformers is great!",
                         "The quick brown fox jumps over the lazy dog." +\
                         "Then the dog got up and ran away because she didn't like foxes.",
                         ],
                         return_tensors="pt",
                         padding=True,
                         truncation=True)
print(f"Token de relleno: {tokenizer.pad_token} | ID del token de relleno: {tokenizer.pad_token_id}")
print("Relleno (padding):")
print_encoding(model_inputs)

In [ ]:
# También puedes decodificar un lote completo a la vez:
print("Decodificación por Lote:")
print(tokenizer.batch_decode(model_inputs.input_ids))
print()
print("Decodificación por Lote: (sin caracteres especiales)")
print(tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens=True))

Para obtener más información sobre los tokenizadores, puedes consultar:
[Hugging Face Transformers Docs](https://huggingface.co/docs/transformers/main_classes/tokenizer) y la [Hugging Face Tokenizers Library](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html).¡La librería de Tokenizers incluso te permite entrenar tus propios tokenizadores!


#### 1.3 Modelos

Inicializar modelos es muy similar a inicializar tokenizadores. Puedes usar la clase del modelo específica para tu modelo o puedes usar una clase AutoModel. Prefiero AutoModel, especialmente cuando quiero comparar modelos, porque es fácil especificar los modelos como cadenas.

Aunque la mayoría de los transformers preentrenados tienen una arquitectura similar, hay pesos adicionales, llamados "heads" (cabecera) que debes entrenar si estás haciendo clasificación de secuencias, preguntas y respuestas, u otra tarea. Hugging Face configura automáticamente la arquitectura que necesitas cuando especificas la clase del modelo. Por ejemplo, estamos haciendo análisis de sentimientos, así que vamos a usar `DistilBertForSequenceClassification`. Si fuéramos a continuar entrenando DistilBERT en su objetivo de entrenamiento de modelado de lenguaje enmascarado, usaríamos `DistilBertForMaskedLM`, y si solo quisiéramos las representaciones del modelo, tal vez para nuestra propia tarea descendente, podríamos usar `DistilBertModel`.

Aquí tienes una imagen estilizada de un modelo recreada a partir de una encontrada aquí: [https://huggingface.co/course/chapter2/2?fw=pt](https://huggingface.co/course/chapter2/2?fw=pt).
![model_illustration.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)

Aquí tienes algunos ejemplos.

`* Model`

`* ForMaskedLM`

`* ForSequenceClassification`

`* ForTokenClassification`

`* ForQuestionAnswering`

`* ForMultipleChoice`

donde `*` puede ser `AutoModel` o un modelo preentrenado específico (por ejemplo, `DistilBert`).

Hay tres tipos de modelos:

* Encoders (por ejemplo, BERT)
* Decoders (por ejemplo, GPT2)
* Modelos Encoder-Decoder (por ejemplo, BART o T5)

Las clases específicas de tareas disponibles dependen del tipo de modelo con el que estés tratando.

Una lista completa de opciones está disponible en los [docs](https://huggingface.co/docs/transformers/model_doc/auto). Ten en cuenta que no todos los modelos son compatibles con todas las arquitecturas de modelos, por ejemplo DistilBERT no es compatible con los modelos Seq2Seq porque solo consiste en un encoder.

In [ ]:
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, DistilBertModel
print('Cargando modelo base')
modelo_base = DistilBertModel.from_pretrained('distilbert-base-cased')
print("Cargando modelo de clasificación desde el checkpoint del modelo base")
modelo = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)
modelo = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

También puedes inicializar con pesos aleatorios.

In [ ]:
from transformers import DistilBertConfig, DistilBertModel

# Inicializando una configuración de DistilBERT
configuracion = DistilBertConfig()
configuracion.num_labels=2
# Inicializando un modelo (con pesos aleatorios) desde la configuración
modelo = DistilBertForSequenceClassification(configuracion)

# Accediendo a la configuración del modelo
configuracion = modelo.config


Pasar entradas al modelo es súper fácil. Este código realiza la inferencia utilizando un modelo de clasificación de secuencias entrenado con un texto tokenizado. Aquí:

- Se convierte la cadena de entrada en tokens y los representa como tensores de PyTorch.
- Los tokens se acompañan de una máscara de atención que indica qué tokens son relevantes.
- Se realiza la inferencia usando los input_ids y attention_mask.
- El modelo produce logits que representan las salidas antes de aplicar la función softmax.
- Se aplica la función softmax a los logits para obtener probabilidades de clase.
- Se interpreta la clase más probable basada en estas probabilidades.

Este flujo de trabajo muestra cómo se utiliza un modelo de clasificación de secuencias preentrenado para hacer predicciones sobre una cadena de texto tokenizada. La salida incluye tanto los tokens de entrada como las predicciones del modelo en forma de logits y distribuciones de probabilidad.


In [ ]:
model_inputs = tokenizer(input_str, return_tensors="pt")
# Opción 1
model_outputs = modelo(input_ids=model_inputs.input_ids, attention_mask=model_inputs.attention_mask)

# Opción 2 - las claves del diccionario que devuelve el tokenizador son las mismas que los argumentos de palabra clave
#            que espera el modelo

# f({k1: v1, k2: v2}) = f(k1=v1, k2=v2)

model_outputs = modelo(**model_inputs)

print(model_inputs)
print()
print(model_outputs)
print()
print(f"Distribución sobre etiquetas: {torch.softmax(model_outputs.logits, dim=1)}")


Si te das cuenta, es un poco extraño que tengamos dos clases para una tarea de clasificación binaria - podrías fácilmente tener una sola clase y simplemente elegir un umbral. Es así por cómo los modelos de huggingface calculan la pérdida. Esto aumentará el número de parámetros que tenemos, pero no debería afectar el rendimiento.

¡Estos modelos son solo módulos de Pytorch! Puedes calcular la pérdida con tu `loss_func` y llamar a `loss.backward`. Puedes usar cualquiera de los optimizadores o planificadores de tasas de aprendizaje que usas.

In [ ]:
# Puedes calcular la pérdida como de costumbre
label = torch.tensor([1])
loss = torch.nn.functional.cross_entropy(model_outputs.logits, label)
print(loss)
loss.backward()
# Puedes obtener los parámetros
list(modelo.named_parameters())[0]

Hugging Face proporciona una forma adicional fácil de calcular la pérdida también:

In [ ]:
# Para calcular la pérdida, necesitamos pasar una etiqueta:
model_inputs = tokenizer(input_str, return_tensors="pt")

labels = ['NEGATIVE', 'POSITIVE']
model_inputs['labels'] = torch.tensor([1])

model_outputs = modelo(**model_inputs)

print(model_outputs)
print()
print(f"Predicciones del modelo: {labels[model_outputs.logits.argmax()]}")

Puedes obtener los estados ocultos y los pesos de atención de los modelos muy fácilmente. Esto es particularmente útil si estás trabajando en un proyecto de análisis. (Por ejemplo, ver [What does BERT look at?](https://arxiv.org/abs/1906.04341)).


El código siguiente carga un modelo preentrenado de distilbert-base-cased utilizando la biblioteca transformers de Hugging Face. Luego, el modelo es utilizado para generar salidas ocultas y atenciones para un texto de entrada.

In [ ]:
from transformers import AutoModel

modelo = AutoModel.from_pretrained("distilbert-base-cased", output_attentions=True, output_hidden_states=True)
modelo.eval()

model_inputs = tokenizer(input_str, return_tensors="pt")
with torch.no_grad():
    model_output = modelo(**model_inputs)

print("Tamaño del estado oculto (por capa):  ", model_output.hidden_states[0].shape)
print("Tamaño del head de atención (por capa):", model_output.attentions[0].shape)     # (capa, lote, índice_palabra_consulta, índices_palabra_clave)
                                                                                       # eje y es consulta, eje x es clave
# print(model_output)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(model_inputs.input_ids[0])
print(tokens)

n_layers = len(model_output.attentions)
n_heads = len(model_output.attentions[0][0])
fig, axes = plt.subplots(6, 12)
fig.set_size_inches(18.5*2, 10.5*2)
for layer in range(n_layers):
    for i in range(n_heads):
        axes[layer, i].imshow(model_output.attentions[layer][0, i])
        axes[layer][i].set_xticks(list(range(9)))
        axes[layer][i].set_xticklabels(labels=tokens, rotation="vertical")
        axes[layer][i].set_yticks(list(range(9)))
        axes[layer][i].set_yticklabels(labels=tokens)

        if layer == 5:
            axes[layer, i].set(xlabel=f"head={i}")
        if i == 0:
            axes[layer, i].set(ylabel=f"layer={layer}")

plt.subplots_adjust(wspace=0.3)
plt.show()

La salida del código es una figura  con múltiples subplots organizados en una cuadrícula de 6 filas y 12 columnas. Cada subplot representa la matriz de atención de una cabecera de atención específica en una capa específica del modelo. En cada matriz de atención:

- Eje X: Representa los tokens de la secuencia de entrada que actúan como claves.
- Eje Y: Representa los tokens de la secuencia de entrada que actúan como consultas.
- Valores: Los valores en la matriz indican cuánta atención está poniendo un token de consulta en cada token de clave. Un valor más alto indica más atención.

Este tipo de visualización es útil para entender cómo el modelo está distribuyendo su atención en diferentes partes de la secuencia de entrada a través de sus múltiples capas y cabeceras de atención.

### Ejercicios

1 . Practica la tokenización y decodificación de texto utilizando AutoTokenizer.

Instrucciones:

- Utiliza el tokenizador distilbert-base-cased para tokenizar una frase de tu elección.
- Imprime los tokens generados.
- Convierte los tokens de nuevo a texto utilizando el método decode.
- Compara el texto original con el texto decodificado y observa cualquier diferencia.

In [ ]:
## Tu respuesta

2 . Utiliza el modelo gpt2 para generar texto basado en una frase inicial dada. Implementa los pasos necesarios para inicializar el modelo y el tokenizer, generar texto y mostrar la salida.

Código de inicio:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Inicializar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained("gpt2")
modelo = AutoModelForCausalLM.from_pretrained("gpt2")

# Generar texto
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generar salida
output = modelo.generate(input_ids, max_length=50, num_return_sequences=1)

# Decodificar y mostrar la salida
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
# Tu respuesta

3 . Utiliza el pipeline de reconocimiento de entidades nombradas (NER) de Hugging Face para extraer entidades de un texto. Implementa los pasos necesarios para inicializar el pipeline, procesar el texto y mostrar las entidades extraídas.

Código de inicio:

In [ ]:
from transformers import pipeline

# Inicializar el pipeline de NER
ner_pipeline = pipeline("ner", grouped_entities=True)

# Texto de ejemplo
text = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge."

# Extraer entidades nombradas
entities = ner_pipeline(text)

# Mostrar las entidades
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")


In [ ]:
# Tu respuesta

4 .  Fine-Tuning de un modelo para una nueva tarea

Ajusta un modelo preentrenado para una nueva tarea de clasificación de sentimientos usando un dataset personalizado.

Instrucciones:
- Carga el dataset yelp_polarity de Hugging Face.
- Ajusta un modelo distilbert-base-uncased usando este dataset.
- Evalúa el modelo ajustado en un conjunto de datos de validación.

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

dataset = load_dataset("yelp_polarity")

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
results = trainer.evaluate()
print(results)


In [ ]:
## Tu respuesta

5 . Visualiza los pesos de atención de un modelo preentrenado.

Instrucciones:

- Carga el modelo bert-base-uncased con la opción output_attentions=True.
- Usa una frase de tu elección y pasa por el modelo para obtener los pesos de atención.
- Visualiza los pesos de atención usando matplotlib.

In [ ]:
from transformers import AutoModel
import matplotlib.pyplot as plt

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
model.eval()

input_str = "El procesamiento de lenguaje natural es fascinante."
model_inputs = tokenizer(input_str, return_tensors="pt")

with torch.no_grad():
    outputs = model(**model_inputs)

attentions = outputs.attentions

# Visualizar la primera capa de atención
fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.matshow(attentions[0][0][0].cpu().numpy(), cmap='viridis')
fig.colorbar(cax)
plt.show()


In [ ]:
# Tu respuesta

6 . Genera texto usando un modelo de lenguaje causal.

Instrucciones:

- Usa el modelo gpt2 para generar texto a partir de una frase inicial.
- Ajusta los parámetros de generación (por ejemplo, max_length, top_p) para observar diferentes resultados.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

prompt = "Era un día soleado cuando"
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(**inputs, max_length=50, do_sample=True, top_p=0.9)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
# Tu respuesta